In [2]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage
from numpy import copy
from skimage import measure, morphology
from os import listdir
from os.path import isfile, join
from skimage import measure, morphology

/home/augt/Public/License2019/clef2019_tuberculosis/.venv/lib/python3.6/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [3]:
def resample(image):
    desiredshape = 130.05504587155963
    real_resize_factor = desiredshape / image.shape[2]
    image = scipy.ndimage.interpolation.zoom(image, [1,1,real_resize_factor])
    
    return image

def resample2(image):
    desireddeep = 130.05504587155963
    real_resize_factor = desireddeep / image.shape[2]
    
    desiredheight = 256
    real_resize_factor_height = desiredheight / image.shape[1]
    
    desiredwidth = 256
    real_resize_factor_width = desiredwidth / image.shape[0]
    
    image = scipy.ndimage.interpolation.zoom(image, [real_resize_factor_height, real_resize_factor_width, real_resize_factor])
    
    return image

def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None

def segment_lung_mask(image, fill_lung_structures=True):
    
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image > -320, dtype=np.int8)+1
    labels = measure.label(binary_image)
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    background_label = labels[0,0,0]
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2
    
    
    # Method of filling the lung structures (that is superior to something like 
    # morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
            
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1

    
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
    
    # Remove other air pockets insided body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image

MIN_BOUND = -1000.0
MAX_BOUND = 400.0

def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

trainDataMean = -1031.3526037498798

def zero_center(image, mean):
    image = image - mean
    return image



In [11]:
firstDataSetPath = "/home/augt/Public/License2019/clef2019_tuberculosis/data/raw/086e42f6-2077-4d8c-a713-6eddd58a4177_TrainingSet_1_of_2/TrainingSet_1_of_2"
secondDataSetPath = "/home/augt/Public/License2019/clef2019_tuberculosis/data/raw/07224578-0898-497a-b21d-94fbf883b974_TrainingSet_2_of_2/TrainingSet_2_of_2"

resampleFolderF16 = "/home/augt/Public/License2019/clef2019_tuberculosis/data/interim/resampledImagesFloat16/"

resampleSegmented = "/home/augt/Public/License2019/clef2019_tuberculosis/data/interim/resampledImagesSegmented/"
resampleSegmentedFill = "/home/augt/Public/License2019/clef2019_tuberculosis/data/interim/resampledImagesSegmentedFill/"
resampleSegmentedDelta = "/home/augt/Public/License2019/clef2019_tuberculosis/data/interim/resampledImagesSegmentedDelta/"

resampleNormalize01F16 = "/home/augt/Public/License2019/clef2019_tuberculosis/data/interim/resampledImagesNormalize01F16/"

meanSumF16 = 0
meanCardinalF16 = 0
meansF16 = []

meanSum2F16 = 0
meanCardinal2F16 = 0
means2F16 = []

#to do-------------------#
#SegmentedNormalize01F16 = "/home/augt/Public/License2019/clef2019_tuberculosis/data/interim/resampledImagesNormalize01F16/"
#DeltaNormalize01F16 = "/home/augt/Public/License2019/clef2019_tuberculosis/data/interim/resampledImagesNormalize01F16/"


for f in listdir(firstDataSetPath):
    if isfile(join(firstDataSetPath, f)):
        auxImage = nib.load(firstDataSetPath + '/' + f);
        #print(auxImage.shape)
        auxImage = resample(auxImage.get_data())
        auxImage = auxImage.astype(np.float16)
        
        normalizedF16 = normalize(auxImage)
        
        ### Get mean of 01 normalized float16 imgs ###
        auxMeanF16 = normalizedF16.mean(dtype=np.float64)
        meanSumF16 += auxMeanF16 * normalizedF16.shape[2]
        meanCardinalF16 += normalizedF16.shape[2]
        meansF16.append(auxMeanF16)
        ##############################################
        
        segmented_lungs = segment_lung_mask(auxImage, False)
        segmented_lungs_fill = segment_lung_mask(auxImage, True)
        segmented_lungs_delta = segmented_lungs_fill - segmented_lungs
        
        np.save(resampleFolderF16 + f, auxImage)
        np.save(resampleNormalize01F16 + f, normalizedF16)
        np.save(resampleSegmented + f, segmented_lungs)
        #np.save(resampleSegmentedFill + f, segmented_lungs_fill)
        np.save(resampleSegmentedDelta + f, segmented_lungs_delta)
        
for f in listdir(secondDataSetPath):
    if isfile(join(secondDataSetPath, f)):
        auxImage = nib.load(secondDataSetPath + '/' + f);
        #print(auxImage.shape)
        auxImage = resample(auxImage.get_data())
        auxImage = auxImage.astype(np.float16)
        
        normalizedF16 = normalize(auxImage)

        
        ### Get mean of 01 normalized float16 imgs ###
        auxMean2F16 = normalizedF16.mean(dtype=np.float64)
        meanSum2F16 += auxMeanF16 * normalizedF16.shape[2]
        meanCardinal2F16 += normalizedF16.shape[2]
        means2F16.append(auxMeanF16)
        ##############################################
        
        segmented_lungs = segment_lung_mask(auxImage, False)
        segmented_lungs_fill = segment_lung_mask(auxImage, True)
        segmented_lungs_delta = segmented_lungs_fill - segmented_lungs
        
        np.save(resampleFolderF16 + f, auxImage)
        np.save(resampleNormalize01F16 + f, normalizedF16)
        np.save(resampleSegmented + f, segmented_lungs)
        #np.save(resampleSegmentedFill + f, segmented_lungs_fill)
        np.save(resampleSegmentedDelta + f, segmented_lungs_delta)
        

In [ ]:
meanF16 = ((meanSumF16/meanCardinalF16)*meanCardinalF16 + (meanSum2F16/meanCardinal2F16)*meanCardinal2F16)
                /(meanCardinalF16 + meanCardinal2F16)

In [ ]:
resampleNormalize01F16 = "/home/augt/Public/License2019/clef2019_tuberculosis/data/interim/resampledImagesNormalize01F16/"

resampleNormalizeMeanF16 = "/home/augt/Public/License2019/clef2019_tuberculosis/data/interim/resampleNormalizeMeanF16/"

#To Do----#

#Should work for task CT Report, because it gets the cleaned of noisy lung#
#SegmentedNormalizeMeanF16 = "/home/augt/Public/License2019/clef2019_tuberculosis/data/interim/resampledImagesNormalizeMeanF16/"

#Should for task Severity Report, because it gets the internal structure of the lung#
#DeltaNormalizeMeanF16 = "/home/augt/Public/License2019/clef2019_tuberculosis/data/interim/resampledImagesNormalizeMeanF16/"

###########
for f in listdir(resampleNormalize01F16):
    if isfile(join(resampleNormalize01F16, f)):
        auxImage = nib.load(resampleNormalize01F16 + '/' + f);
        auxImage = zero_center(auxImage, meanF16)
        np.save(resampleNormalizeMeanF16 + f, auxImage)


In [7]:
########## DATA POOLED TO 256 256 ###################
firstDataSetPath = "/home/augt/Public/License2019/clef2019_tuberculosis/data/raw/086e42f6-2077-4d8c-a713-6eddd58a4177_TrainingSet_1_of_2/TrainingSet_1_of_2"
secondDataSetPath = "/home/augt/Public/License2019/clef2019_tuberculosis/data/raw/07224578-0898-497a-b21d-94fbf883b974_TrainingSet_2_of_2/TrainingSet_2_of_2"

Normalize01F16MaxPool = "/home/augt/Public/License2019/clef2019_tuberculosis/data/interim/Train_Normalize01F16MaxPool/"


for f in listdir(firstDataSetPath):
    if isfile(join(firstDataSetPath, f)):
        auxImage = nib.load(firstDataSetPath + '/' + f);
        #print(auxImage.shape)
        auxImage = resample2(auxImage.get_data())
        auxImage = auxImage.astype(np.float16)
        
        normalizedF16 = normalize(auxImage)
        
        np.save(Normalize01F16MaxPool + f, normalizedF16)
        
for f in listdir(secondDataSetPath):
    if isfile(join(secondDataSetPath, f)):
        auxImage = nib.load(secondDataSetPath + '/' + f);
        #print(auxImage.shape)
        auxImage = resample2(auxImage.get_data())
        auxImage = auxImage.astype(np.float16)
        
        normalizedF16 = normalize(auxImage)
        
        np.save(Normalize01F16MaxPool + f, normalizedF16)


        

        

In [7]:


########## DATA POOLED TO 256 256 ###################
trainSet = "/home/augt/Public/License2019/clef2019_tuberculosis/data/raw/8b238bbc-bad1-4637-b12c-246728298a49_TestSet/TestSet"

Normalize01F16MaxPool = "/home/augt/Public/License2019/clef2019_tuberculosis/data/processed/Train_Normalize01F16MaxPool/"


for f in listdir(trainSet):
    if isfile(join(trainSet, f)):
        auxImage = nib.load(trainSet + '/' + f);
        #print(auxImage.shape)
        auxImage = resample2(auxImage.get_data())
        auxImage = auxImage.astype(np.float16)
        
        normalizedF16 = normalize(auxImage)
        
        np.save(Normalize01F16MaxPool + f, normalizedF16)

